In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import re
import random

from IPython.display import display

from nltk import word_tokenize, sent_tokenize

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV,StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, classification_report
from sklearn.dummy import DummyClassifier
import nltk

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_url = 'https://media.githubusercontent.com/media/ricadolcvuba/Dataset-TP2/main/df_train_preprocesado.csv'
df_train = pd.read_csv(train_url)
x_train = df_train['review_es']
y_train = df_train['sentimiento']
test_url = 'https://media.githubusercontent.com/media/ricadolcvuba/Dataset-TP2/main/df_test_preprocesado.csv'
df_test = pd.read_csv(test_url)
x_test = df_test['review_es']

#Vectorizar

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
x_train_text = df_train['review_es']
y_train_text = df_train['sentimiento']

x_test_text = df_test['review_es']

In [ ]:
my_stop_words = stopwords.words('spanish')

# Instanciamos el CV
#Configurar el vectorizador de palabras para convertir el texto en una representación numérica.
vec = CountVectorizer(stop_words=my_stop_words, ngram_range=(1,2))

# Ajustamos el CV y transformamos los datos
#transformar los datos de entrenamiento en una matriz numérica.
x_train_aux = vec.fit_transform(x_train_text)
#Transformar los datos de prueba en la misma representación numérica que el conjunto de entrenamiento
x_test = vec.transform(x_test_text)

#Ensamble (Voting)

In [ ]:
from sklearn.ensemble import VotingClassifier

##XGBoost

In [ ]:
model_xgb = joblib.load('/content/drive/MyDrive/modelos/xgboost_n10cv10.joblib')

##Random forest

In [ ]:
model_rf = joblib.load('/content/drive/MyDrive/modelos/rf_n10cv10.joblib')

##RN

In [ ]:
model_rn = joblib.load('/content/drive/MyDrive/modelos/best_nb.joblib')

##Voting con Red neuronal, XGBoost y Random Forest

In [ ]:
voting_rn_xgb_rf = VotingClassifier(estimators = [('xgb', model_xgb), ('rn', model_rn), ('rf', model_rf)], voting = 'hard')

In [ ]:
voting_rn_xgb_rf.fit(x_train_aux,y_train)

VotingClassifier(estimators=[('xgb',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eta=0.6000000000000001,
                                            eval_metric=None,
                                            feature_types=None,
                                            gamma=5.684210526315789,
                                            grow_policy=None,
                                            importance_type=None,
                                            inte...
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=120, n_jobs=None, ...)),
                             ('rn',
                              MultinomialNB(alpha=1.0714285714285714,
                                            class_prior=[0.2, 0.8],
                                            fit_prior=False,
                                            force_alpha=True)),
                             ('rf',
                              RandomForestClassifier(criterion='entropy',
                                                     min_samples_leaf=5,
                                                     min_samples_split=14,
                                                     n_estimators=57, n_jobs=-1,
                                                     oob_score=True,
                                                     random_state=1))])

In [ ]:
joblib.dump(voting_rn_xgb_rf, '/content/drive/MyDrive/modelos/voting_rn_xgb_rf.joblib')

['/content/drive/MyDrive/modelos/voting_rn_xgb_rf.joblib']

In [ ]:
voting_rn_xgb_rf

VotingClassifier(estimators=[('xgb',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eta=0.6000000000000001,
                                            eval_metric=None,
                                            feature_types=None,
                                            gamma=5.684210526315789,
                                            grow_policy=None,
                                            importance_type=None,
                                            inte...
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=120, n_jobs=None, ...)),
                             ('rn',
                              MultinomialNB(alpha=1.0714285714285714,
                                            class_prior=[0.2, 0.8],
                                            fit_prior=False,
                                            force_alpha=True)),
                             ('rf',
                              RandomForestClassifier(criterion='entropy',
                                                     min_samples_leaf=5,
                                                     min_samples_split=14,
                                                     n_estimators=57, n_jobs=-1,
                                                     oob_score=True,
                                                     random_state=1))])

In [ ]:
y_pred_voting_rn_xgb_rf = voting_rn_xgb_rf.predict(x_test)

In [ ]:
df_pred_voting = pd.DataFrame()
df_pred_voting['ID'] = df_test['ID']
df_pred_voting['sentimiento'] = y_pred_voting_rn_xgb_rf

df_pred_voting.to_csv('/content/drive/MyDrive/pred/y_pred_voting_rn_xgb_rf.csv', index=False)